# LlamaIndex Agent Reasoning Loop

- Function calling means the LLM is outputting structured format for the agent to call functions.
- Suggesting you only use `GPT-3.5-Turbo` or `GPT-4` for function calling.

In [2]:
import nest_asyncio

nest_asyncio.apply()
# !wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

In [3]:
from typing import List, Optional

from llama_index.core.indices import SummaryIndex, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition


def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(query: str, page_numbers: Optional[List[str]] = None) -> str:
        """Use to answer questions over a given paper.

        Useful if you have specific questions over the paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts, condition=FilterCondition.OR
            ),
        )
        response = query_engine.query(query)
        return response

    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}", fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(f"Useful for summarization questions related to {name}"),
    )

    return vector_query_tool, summary_tool

In [4]:
import os

from llama_index.core import Settings, SummaryIndex, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI

AZURE_API_ENDPOINT = os.environ.get("AZURE_API_ENDPOINT")
AZURE_API_KEY = os.environ.get("AZURE_API_KEY")

llm = AzureOpenAI(
    model="gpt-4-32k",
    api_version="2024-02-01",
    azure_endpoint=AZURE_API_ENDPOINT,
    api_key=AZURE_API_KEY,
    deployment_name="gpt-4-32k",
    temperature=0,
)
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    azure_endpoint=AZURE_API_ENDPOINT,
    api_key=AZURE_API_KEY,
    api_version="2024-02-01",
    deployment_name="text-embedding-3-small",
)

Settings.llm = llm
Settings.embed_model = embed_model

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

In [5]:
from llama_index.core.agent import AgentRunner, FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True,
)
agent = AgentRunner(agent_worker)

In [6]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, various roles are assigned to different agents, such as Product Manager, Architect, and Engineer. Each agent has a specific role and expertise, and they follow established standards. This allows for automatic requirement analysis, system design, code generation, modification, execution, and debugging during runtime. The addition of different roles, apart from just the Engineer, consistently improves both revisions and executability of the code. While more roles slightly increase the expenses, the overall performance improves noticeably.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "communication between agents in MetaGPT"}
=== Function Output ===
In the MetaGPT framework, agents can coll

In [7]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 2
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-22
last_modified_date: 2024-06-22

Preprint
Figure 1: The software development SOPs between MetaGPT and real-world human teams.
In software engineering, SOPs promote collaboration among various roles. MetaGPT showcases
its ability to decompose complex tasks into specific actionable procedures assigned to various roles
(e.g., Product Manager, Architect, Engineer, etc.).
documents, design artifacts, flowcharts, and interface specifications. The use of intermediate struc-
tured outputs significantly increases the success rate of target code generation. Because it helps
maintain consistency in communication, minimizing ambiguities and errors during collaboration.
More graphically, in a company simulated by MetaGPT, all employees follow a strict and stream-
lined workflow, and all their handovers must comply with certain established standards. This reduces
the ri

In [8]:
response = agent.chat("Tell me about the evaluation datasets used.")

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include the SoftwareDev dataset, HumanEval, and MBPP benchmarks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include:

1. The SoftwareDev dataset: This dataset is used to evaluate the performance of the model in software development tasks.

2. HumanEval: This is another dataset used for evaluation. It consists of a variety of programming tasks and is used to assess the model's ability to solve these tasks.

3. MBPP benchmarks: These benchmarks are also used in the evaluation process to measure the model's performance.


In [9]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results of MetaGPT on the SoftwareDev dataset"}
=== Function Output ===
MetaGPT has shown impressive results on the SoftwareDev dataset. It achieved an average score of 3.9, surpassing ChatDev's score of 2.1. In terms of executability, MetaGPT scored 3.75, which is very close to 4 (flawless). It also took less time to run, with a time of 503 seconds, which is significantly less than ChatDev. Although MetaGPT required more tokens (24,613 or 31,255 compared to 19,292), it only needed 126.5/124.3 tokens to generate one line of code, while ChatDev used 248.9 tokens. The cost of human revision for MetaGPT was also lower. Furthermore, MetaGPT demonstrated superior performance when used with GPT-4 as the backend on selected SoftwareDev tasks.
=== LLM Response ===
MetaGPT has shown impressive results on the SoftwareDev dataset. I

## Lower-Level: Debuggability and Control

In [10]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [11]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [12]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, various roles are assigned to different agents, such as Product Manager, Architect, Engineer, and others. These roles help in decomposing complex tasks into specific actionable procedures. Each agent has a specific role and expertise, following established standards. This allows for automatic requirement analysis, system design, code generation, modification, execution, and debugging during runtime. The addition of roles different from just the Engineer consistently improves both revisions and executability. While more roles slightly increase the expenses, the overall performance improves noticeably.


In [13]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 7b34a62a-dcc5-4d7e-b0b2-6591ff920244: 1
In MetaGPT, various roles are assigned to different agents, such as Product Manager, Architect, Engineer, and others. These roles help in decomposing complex tasks into specific actionable procedures. Each agent has a specific role and expertise, following established standards. This allows for automatic requirement analysis, system design, code generation, modification, execution, and debugging during runtime. The addition of roles different from just the Engineer consistently improves both revisions and executability. While more roles slightly increase the expenses, the overall performance improves noticeably.


In [14]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 7b34a62a-dcc5-4d7e-b0b2-6591ff920244: 1


TaskStep(task_id='7b34a62a-dcc5-4d7e-b0b2-6591ff920244', step_id='7bac4211-f033-4807-99b3-0b479aaf1b35', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [15]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "communication between agents in MetaGPT"}
=== Function Output ===
In the MetaGPT framework, communication between agents is facilitated through defined development and communication protocols. This allows for collaboration among various agents, enabling them to work together to accomplish more complex tasks or projects.


In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, the agents communicate with each other through established development and communication protocols. These protocols enable the agents to collaborate effectively, allowing them to work together to accomplish more complex tasks or projects. This communication is crucial for the functioning of the system as it allows for the decomposition of complex tasks into specific actionable procedures, which are then handled by the agents according to their specific roles and expertise.
True


In [17]:
response = agent.finalize_response(task.task_id)

In [18]:
print(str(response))

In MetaGPT, the agents communicate with each other through established development and communication protocols. These protocols enable the agents to collaborate effectively, allowing them to work together to accomplish more complex tasks or projects. This communication is crucial for the functioning of the system as it allows for the decomposition of complex tasks into specific actionable procedures, which are then handled by the agents according to their specific roles and expertise.
